<a href="https://colab.research.google.com/github/KillerGlass/classifica-o-binaria-em-pytroch/blob/main/Classifica%C3%A7%C3%A3o_binaria_com_tuning_dos_parametros_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Etapa 1 : Importação das bibliotecas

In [ ]:
!pip install skorch==0.8.0
#pytroch n tem suporte para validação cruzada

     |████████████████████████████████| 113 kB 12.2 MB/s 


In [ ]:
import pandas as pd
from sklearn.model_selection import  GridSearchCV
import numpy as np
from skorch import NeuralNetBinaryClassifier
import torch
import torch.nn as nn
import torch.nn.functional as F

import seaborn as sns
import skorch
torch.__version__

'1.9.0+cu111'

# Etapa 2 : Base de dados

In [ ]:
np.random.seed(123)
torch.manual_seed(123)

In [ ]:
prev = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep Learnig/Bases/entradas_breast.csv')
target = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep Learnig/Bases/saidas_breast.csv')

In [ ]:
prev = np.array(prev, dtype='float32')
target = np.array(target, dtype='float32').squeeze(1)

# Etapa 3 : Classe para estrutura da rede neural

In [ ]:
class classficador_torch(nn.Module):
#diferente dos outros projetos, nesse usaremos o tuning nos parametros
#que nada mais que passar uma lista com comandos e o algoritmo escolher qual é o melhor a ser usado

  def __init__(self,activation,neurons,initializer):
    super().__init__()
    #todos os parametros serao decididos de maneira dinamica
    self.densa0 = nn.Linear(30,neurons)
    initializer(self.densa0.weight) #iniciando os pesos de maneira dinamica
    self.activate0 = activation

    #segunda camada
    self.densa1 = nn.Linear(neurons,neurons)
    initializer(self.densa1.weight) 
    self.activate1 = activation

    #terceira camada
    self.densa2 = nn.Linear(neurons,1)
    initializer(self.densa2.weight)  
    #a ultima camada de ativação n sera alterada
    self.output = nn.Sigmoid()

  def forward(self,x):

    x = self.densa0(x)
    x = self.activate0(x)

    x = self.densa1(x)
    x = self.activate1(x)

    x = self.densa2(x)
    x = self.output(x)

    return x


# Etapa 4: Skorch

In [ ]:
#não sao passados parametros de otimização e outros
class_sklearn = NeuralNetBinaryClassifier(module= classficador_torch,
                                          lr=0.001,
                                          optimizer__weight_decay=0.0001,
                                          train_split=False)

#Etapa 5: Tuning dos parametros

In [ ]:
#parametros a se passar
#melhor a se fazer e testar cada um dos parametros
parans = {"batch_size": [10],
          "max_epochs":[100],
          "optimizer":[torch.optim.Adam,torch.optim.SGD],
          "criterion": [torch.nn.BCELoss, torch.nn.HingeEmbeddingLoss],#torch.optim.SGB],
          "module__activation": [F.relu, F.tanh],
          "module__neurons":[8,16],
          "module__initializer":[torch.nn.init.uniform_]}#torch.nn.init.normal]}

In [ ]:
parans

{'batch_size': [10],
 'criterion': [torch.nn.modules.loss.BCELoss,
  torch.nn.modules.loss.HingeEmbeddingLoss],
 'max_epochs': [100],
 'module__activation': [<function torch.nn.functional.relu>,
  <function torch.nn.functional.tanh>],
 'module__initializer': [<function torch.nn.init.uniform_>],
 'module__neurons': [8, 16],
 'optimizer': [torch.optim.adam.Adam, torch.optim.sgd.SGD]}

In [ ]:
grid_search = GridSearchCV(estimator=class_sklearn, param_grid=parans,
                           scoring='accuracy', cv=2)
grid_search = grid_search.fit(prev,target)

  epoch    train_loss     dur
-------  ------------  ------
      1       37.3239  0.0281
      2       37.3239  0.0256
      3       37.3239  0.0266
      4       37.3239  0.0263
      5       37.3239  0.0307
      6       37.3239  0.0255
      7       37.3239  0.0296
      8       37.3239  0.0252
      9       37.3239  0.0257
     10       37.3239  0.0264
     11       37.3239  0.0267
     12       37.3239  0.0262
     13       37.3239  0.0276
     14       37.3239  0.0274
     15       37.3239  0.0290
     16       37.3239  0.0295
     17       37.3239  0.0296
     18       37.3239  0.0286
     19       37.3239  0.0268
     20       37.3239  0.0273
     21       37.3239  0.0263
     22       37.3239  0.0301
     23       37.3239  0.0285
     24       37.3239  0.0247
     25       37.3239  0.0267
     26       37.3239  0.0280
     27       37.3239  0.0286
     28       37.3239  0.0242
     29       37.3239  0.0257
     30       37.3239  0.0282
     31       37.3239  0.0264
     32   

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


      7        1.0247  0.0305
      8        0.9451  0.0334
      9        0.8507  0.0252
     10        0.7591  0.0270
     11        0.7032  0.0280
     12        0.6809  0.0261
     13        0.6732  0.0287
     14        0.6705  0.0271
     15        0.6693  0.0258
     16        0.6687  0.0260
     17        0.6683  0.0260
     18        0.6680  0.0263
     19        0.6677  0.0255
     20        0.6676  0.0296
     21        0.6674  0.0290
     22        0.6673  0.0322
     23        0.6672  0.0270
     24        0.6671  0.0269
     25        0.6670  0.0271
     26        0.6670  0.0254
     27        0.6669  0.0261
     28        0.6668  0.0282
     29        0.6668  0.0273
     30        0.6668  0.0269
     31        0.6667  0.0274
     32        0.6667  0.0300
     33        0.6666  0.0315
     34        0.6666  0.0304
     35        0.6666  0.0267
     36        0.6665  0.0261
     37        0.6665  0.0254
     38        0.6665  0.0276
     39        0.6665  0.0296
     40   

In [ ]:
melhores = grid_search.best_params_
precis = grid_search.best_score_

In [ ]:
melhores

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.BCELoss,
 'max_epochs': 100,
 'module__activation': <function torch.nn.functional.relu>,
 'module__initializer': <function torch.nn.init.uniform_>,
 'module__neurons': 16,
 'optimizer': torch.optim.adam.Adam}

In [ ]:
precis

0.7365702989869038